# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("Output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mejit,10.2753,170.8646,27.58,81,100,7.88,MH,1715420985
1,1,aldama,22.9167,-98.0667,25.76,90,41,1.39,MX,1715420985
2,2,badger,64.8000,-147.5333,6.15,52,0,3.09,US,1715420986
3,3,kitimat,54.0524,-128.6534,11.21,70,97,0.56,CA,1715420987
4,4,albany,42.6001,-73.9662,9.40,94,100,1.13,US,1715420988


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) &
                                    (city_data_df["Max Temp"] < 27) &
                                    (city_data_df["Wind Speed"] < 4.5) &
                                    (city_data_df["Cloudiness"] == 0), :]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how="any")
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,93,yoloten,37.2989,62.3597,24.42,27,0,2.77,TM,1715421069
137,137,namibe,-15.1961,12.1522,25.96,66,0,3.10,AO,1715421104
162,162,ploudalmezeau,48.5333,-4.6500,23.21,49,0,3.09,FR,1715421126
164,164,colonia,50.9333,6.9500,23.58,49,0,1.54,DE,1715421127
169,169,gazojak,41.1875,61.4036,26.32,19,0,1.81,TM,1715421133
171,171,talant,47.3369,5.0089,21.19,64,0,2.06,FR,1715421135
192,192,la couronne,45.6113,0.0995,25.31,43,0,2.57,FR,1715421154
274,274,yanchep,-31.5500,115.6833,22.07,85,0,3.18,AU,1715421227
371,371,ormond-by-the-sea,29.3491,-81.0665,24.05,89,0,2.06,US,1715421309
412,412,anapolis,-16.3267,-48.9528,26.78,47,0,2.06,BR,1715421345


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df.drop(columns=["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
93,yoloten,37.2989,62.3597,27,TM,
137,namibe,-15.1961,12.1522,66,AO,
162,ploudalmezeau,48.5333,-4.6500,49,FR,
164,colonia,50.9333,6.9500,49,DE,
169,gazojak,41.1875,61.4036,19,TM,
171,talant,47.3369,5.0089,64,FR,
192,la couronne,45.6113,0.0995,43,FR,
274,yanchep,-31.5500,115.6833,85,AU,
371,ormond-by-the-sea,29.3491,-81.0665,89,US,
412,anapolis,-16.3267,-48.9528,47,BR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yoloten - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
ploudalmezeau - nearest hotel: Hostellerie du Castel
colonia - nearest hotel: Wasserturm Hotel Cologne
gazojak - nearest hotel: Жайхун
talant - nearest hotel: La Bonbonnière
la couronne - nearest hotel: Hôtel Marjolaine
yanchep - nearest hotel: Yanchep Inn
ormond-by-the-sea - nearest hotel: Howard Johnson
anapolis - nearest hotel: Hotel UAI
recife - nearest hotel: No hotel found
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
xianyang - nearest hotel: 如家精选酒店(西安沣西港沣国际店)


,City,Lat,Lng,Humidity,Country,Hotel Name
93,yoloten,37.2989,62.3597,27,TM,No hotel found
137,namibe,-15.1961,12.1522,66,AO,Nelsal Pensao
162,ploudalmezeau,48.5333,-4.6500,49,FR,Hostellerie du Castel
164,colonia,50.9333,6.9500,49,DE,Wasserturm Hotel Cologne
169,gazojak,41.1875,61.4036,19,TM,Жайхун
171,talant,47.3369,5.0089,64,FR,La Bonbonnière
192,la couronne,45.6113,0.0995,43,FR,Hôtel Marjolaine
274,yanchep,-31.5500,115.6833,85,AU,Yanchep Inn
371,ormond-by-the-sea,29.3491,-81.0665,89,US,Howard Johnson
412,anapolis,-16.3267,-48.9528,47,BR,Hotel UAI


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
# %%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = 'all'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)